#### Import Necessary Libraries

In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import dbnomics
from dbnomics import fetch_series, fetch_series_by_api_link

adbCodes=pd.read_csv(os.path.join(os.getcwd(),"data","ADB_CountryCodes.csv"))

### Download Data from Trading Economics

In [26]:
#Set Necessary headers to read data
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
URL="https://tradingeconomics.com/"+adbCodes['TE_Name'].iloc[0]+"/currency"
response=requests.get(URL,headers=headers)
soup=BeautifulSoup(response.content,'html.parser')
tables=soup.find_all('table','table')

In [21]:
"USD"+adbCodes['CurrencyCode'].iloc[0]

'USDAFN'

In [46]:
df1=pd.read_html(str(tables[1]))[0]
price=float(df1[df1['Crosses']=="USD"+adbCodes['CurrencyCode'].iloc[0]]['Price'])
YoYChange=df1[df1['Crosses']=="USD"+adbCodes['CurrencyCode'].iloc[0]]['Year'][0]


In [48]:
df1

,Crosses,Price,Unnamed: 2,Unnamed: 3,Day,Year,Date
0,USDAFN,88.8800,NaN,0.00000,0%,13.95%,Jun/06
1,EURAFN,95.2705,NaN,-0.27830,-0.29%,0.62%,Jun/03
2,GBPAFN,110.9916,NaN,-0.79210,-0.71%,0.78%,Jun/03
3,AUDAFN,64.2229,NaN,0.17690,0.28%,6.82%,Jun/06
4,NZDAFN,57.8156,NaN,-0.49010,-0.84%,3.61%,Jun/03
5,AFNJPY,1.4716,NaN,-0.00020,-0.01%,4.35%,Jun/06
6,AFNCNY,0.0748,NaN,-0.00010,-0.07%,-9.09%,Jun/06
7,AFNCHF,0.0108,NaN,-0.00001,-0.08%,-6.50%,Jun/06
8,AFNCAD,0.0141,NaN,-0.00003,-0.22%,-9.10%,Jun/06
9,AFNMXN,0.2194,NaN,-0.00050,-0.25%,-14.56%,Jun/06
